### Distributed MCMC Retrieval

This notebook runs the MCMC retrievals on a local cluster using `ipyparallel`.


In [3]:
import ipyparallel as ipp
c = ipp.Client(profile='gold')
lview = c.load_balanced_view()

## Retrieval Setup

In [2]:
%%px
%env ARTS_BUILD_PATH=/home/simonpf/build/arts
%env ARTS_INCLUDE_PATH=/home/simonpf/src/atms_simulations/:/home/simonpf/src/arts/controlfiles
%env ARTS_DATA_PATH=/home/simonpf/src/arts_xml/
%env OMP_NUM_THREADS=1

import sys
sys.path.insert(1,"/home/simonpf/src/atms_simulations/")
sys.path.insert(1, "/home/simonpf/src/typhon/")

import os
os.chdir("/home/simonpf/src/atms_simulations")

# This is important otherwise engines just crash.
import matplotlib; matplotlib.use("agg")

NoEnginesRegistered: Can't build targets without any engines

In [5]:
from typhon.arts.workspace import Workspace
import atms
import numpy as np

ws = Workspace()
channels = [0,15,16,17,19]
atms.setup_atmosphere(ws)
atms.setup_sensor(ws, channels)
atms.checks(ws)
ws.yCalc()

Loading ARTS API from: /home/simonpf/build/arts/src/libarts_api.so
ARTS[43647152]: Executing /home/simonpf/src/arts/controlfiles/general/general.arts
ARTS[43647152]: {
ARTS[43647152]: - verbosityInit
ARTS[43647152]: - scat_speciesSet
ARTS[43647152]: - MatrixSet
ARTS[43647152]: - Tensor4SetConstant
ARTS[43647152]: - ArrayOfStringSet
ARTS[43647152]: - Touch
ARTS[43647152]: - FlagOff
ARTS[43647152]: - MatrixSet
ARTS[43647152]: - NumericSet
ARTS[43647152]: - ArrayOfStringSet
ARTS[43647152]: - Tensor3SetConstant
ARTS[43647152]: - Tensor3SetConstant
ARTS[43647152]: - Tensor3SetConstant
ARTS[43647152]: - Tensor3SetConstant
ARTS[43647152]: - Tensor3SetConstant
ARTS[43647152]: - Tensor3SetConstant
ARTS[43647152]: - IndexSet
ARTS[43647152]: - IndexSet
ARTS[43647152]: - IndexSet
ARTS[43647152]: - IndexSet
ARTS[43647152]: - FlagOff
ARTS[43647152]: - output_file_formatSetAscii
ARTS[43647152]: - StringSet
ARTS[43647152]: - IndexSet
ARTS[43647152]: - abs_lineshapeDefine
ARTS[43647152]: - NumericSet
A

In [6]:
%%px
from typhon.arts.workspace import Workspace
import atms
import numpy as np

ws = Workspace()
channels = [0,15,16,17,19]
atms.setup_atmosphere(ws)
atms.setup_sensor(ws, channels)
atms.checks(ws)
ws.yCalc()

[stdout:0] 
Loading ARTS API from: /home/simonpf/build/arts/src/libarts_api.so
ARTS[47768304]: Executing /home/simonpf/src/arts/controlfiles/general/general.arts
ARTS[47768304]: {
ARTS[47768304]: - verbosityInit
ARTS[47768304]: - scat_speciesSet
ARTS[47768304]: - MatrixSet
ARTS[47768304]: - Tensor4SetConstant
ARTS[47768304]: - ArrayOfStringSet
ARTS[47768304]: - Touch
ARTS[47768304]: - FlagOff
ARTS[47768304]: - MatrixSet
ARTS[47768304]: - NumericSet
ARTS[47768304]: - ArrayOfStringSet
ARTS[47768304]: - Tensor3SetConstant
ARTS[47768304]: - Tensor3SetConstant
ARTS[47768304]: - Tensor3SetConstant
ARTS[47768304]: - Tensor3SetConstant
ARTS[47768304]: - Tensor3SetConstant
ARTS[47768304]: - Tensor3SetConstant
ARTS[47768304]: - IndexSet
ARTS[47768304]: - IndexSet
ARTS[47768304]: - IndexSet
ARTS[47768304]: - IndexSet
ARTS[47768304]: - FlagOff
ARTS[47768304]: - output_file_formatSetAscii
ARTS[47768304]: - StringSet
ARTS[47768304]: - IndexSet
ARTS[47768304]: - abs_lineshapeDefine
ARTS[47768304]: - 

## A Priori State

The simulations are based on the a priori assumptions, that the profiles of specific humidity, temperature and ozone vary independently and that the relative variations can be described by Log-Gaussian distributions.

In [7]:
%%px
qt_mean = np.load("data/qt_mean.npy").ravel()
qt_cov     = np.load("data/qt_cov.npy")
qt_cov_inv  = np.linalg.inv(qt_cov)

## Jumping Functions

The jumping functions are used inside the MCMC iteration and propose new atmospheric states for specific humidity, temperature and ozone, respectively. The proposed states are generated from random walks that use scaled versions of the a priori covariances.

In [8]:
%%px
import numpy as np
from typhon.retrieval.mcmc import RandomWalk
c = (1.0 / np.sqrt(qt_mean.size)) ** 2
rw_qt  = RandomWalk(c * qt_cov)

def j_qt(ws, x, revert = False):
    if revert:
        x_new = x
    else:
        x_new = rw_qt.step(x)
    q_new = (np.exp(x_new[14::-1]).reshape((15,)))
    q_new = atms.mmr2vmr(ws, q_new, "h2o")
    ws.vmr_field.value[0, :, 0, 0] = q_new
    ws.t_field.value[:, 0, 0] = x_new[:14:-1]
    ws.sst = np.maximum(ws.t_field.value[0, 0, 0], 270.0)
    return x_new

## A Priori Distributions

These functions return the likelihood (up to an additive constant) of a given state for each of the variables. Note that the states of specific humidity, temperature and ozone are given by the logs of the relative variations. 

In [9]:
%%px
def p_a_qt(x):
    dx = x - qt_mean
    l = - 0.5 * np.dot(dx, np.dot(qt_cov_inv, dx))
    return l

## Measurement Uncertainty

We assume that uncertainty of the measured brightness temperatures can be described by independent Gaussian error with a standard deviation of $1 K$.

In [10]:
%%px
covmat_y = np.diag(np.ones(len(channels)))
covmat_y_inv = np.linalg.inv(covmat_y)

def p_y(y, yf):
    dy = y - yf
    l  = - 0.5 * np.dot(dy, np.dot(covmat_y_inv, dy))
    return l

# Running MCMC


### The Simulated Measurement

For the simulated measurement, we sample a state from the a priori distribution of atmsopheric states and simulate the measured brightness temperatures.

A simple heuristic is applied to ensure that reasonable acceptance rates are obtained during the MCMC simulations. After the initial burn-in phase, 1000 simulation steps are performed. If the acceptance rates during this simulation are too low/high that covariance matrices of the corresponding random walks are scaled by a factor 0.1 / 9.0, respectively.

In [11]:
%%px
def adapt_covariances(a):
    if (np.sum(a[:, 0]) / a.shape[0]) < 0.2:
        rw_qt.covmat *= 0.7
    if (np.sum(a[:, 0]) / a.shape[0]) > 0.4:
        rw_qt.covmat *= 1.5

In [12]:
%%px
from typhon.retrieval.mcmc import MCMC
from atms import vmr2cd

dist = atms.StateDistribution()
n_burn_in = 500
n_prod    = 5000
drop      = 10

In [13]:
from typhon.retrieval.mcmc import MCMC
from atms import vmr2cd

def run_retrieval(i):
   
    # Reset covariance matrices.
    rw_qt.covmat = np.copy(c * qt_cov)

    # Generate True State
    dist.sample(ws)
    ws.yCalc()
    y_true   = np.copy(ws.y)
    q_true  = np.copy(ws.vmr_field.value[0, :, 0, 0].ravel())
    t_true  = np.copy(ws.t_field.value[:, 0, 0].ravel())
    cwv_true = atms.vmr2cd(ws)
    
    dist.a_priori(ws)
    qt = np.zeros(qt_mean.size)

    # Add Noise
    y_true += np.random.randn(*y_true.shape)
    
    #try:
    mcmc  = MCMC([[qt, p_a_qt, j_qt]], y_true, p_y, [vmr2cd])

    qt_0 = dist.sample_factors()
    _, _, _, a = mcmc.warm_up(ws, [qt_0], n_burn_in)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, n_burn_in)
    hist_1, s_1, _, _ = mcmc.run(ws, n_prod)

    # Reset covariance matrices.
    rw_qt.covmat = np.copy(c * qt_cov)
    qt_0 = dist.sample_factors()
    _, _, _, a = mcmc.warm_up(ws, [qt_0], n_burn_in)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, n_burn_in)
    hist_2, s_2, _, _ = mcmc.run(ws, n_prod)

    # Reset covariance matrices.
    rw_qt.covmat = np.copy(c * qt_cov)
    qt_0 = dist.sample_factors()
    _, _, _, a = mcmc.warm_up(ws, [qt_0], n_burn_in)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, n_burn_in)
    hist_3, s_3, _, _ = mcmc.run(ws, n_prod)

    # Reset covariance matrices.
    rw_qt.covmat = np.copy(c * qt_cov)
    qt_0 = dist.sample_factors()
    _, _, _, a = mcmc.warm_up(ws, [qt_0], n_burn_in)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, n_burn_in)
    hist_4, s_4, _, _ = mcmc.run(ws, n_prod)

    # Reset covariance matrices.
    rw_qt.covmat = np.copy(c * qt_cov)
    qt_0 = dist.sample_factors()
    _, _, _, a = mcmc.warm_up(ws, [qt_0], n_burn_in)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, n_burn_in)
    hist_5, s_5, _, _ = mcmc.run(ws, n_prod)

    # Reset covariance matrices.
    rw_qt.covmat = np.copy(c * qt_cov)
    qt_0 = dist.sample_factors()
    _, _, _, a = mcmc.warm_up(ws, [qt_0], n_burn_in)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, n_burn_in)
    hist_6, s_6, _, _ = mcmc.run(ws, n_prod)

    # Reset covariance matrices.
    rw_qt.covmat = np.copy(c * qt_cov)
    qt_0 = dist.sample_factors()
    _, _, _, a = mcmc.warm_up(ws, [qt_0], n_burn_in)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, n_burn_in)
    hist_7, s_7, _, _ = mcmc.run(ws, n_prod)

    # Reset covariance matrices.
    rw_qt.covmat = np.copy(c * qt_cov)
    qt_0 = dist.sample_factors()
    _, _, _, a = mcmc.warm_up(ws, [qt_0], n_burn_in)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, 200)
    adapt_covariances(a)
    _, _, _, a = mcmc.run(ws, n_burn_in)
    hist_8, s_8, _, _ = mcmc.run(ws, n_prod)

    profiles_q =  np.stack([hist_1[0][::drop, :15],
                            hist_2[0][::drop, :15],
                            hist_3[0][::drop, :15],
                            hist_4[0][::drop, :15],
                            hist_5[0][::drop, :15],
                            hist_6[0][::drop, :15],
                            hist_7[0][::drop, :15],
                            hist_8[0][::drop, :15]])
    profiles_t =  np.stack([hist_1[0][::drop, 15:],
                            hist_2[0][::drop, 15:],
                            hist_3[0][::drop, 15:],
                            hist_4[0][::drop, 15:],
                            hist_5[0][::drop, 15:],
                            hist_6[0][::drop, 15:],
                            hist_7[0][::drop, 15:],
                            hist_8[0][::drop, 15:]])
    cwv = np.stack([s_1[::drop], s_2[::drop], s_3[::drop], s_4[::drop],
                    s_5[::drop],s_6[::drop],s_7[::drop],s_8[::drop]], axis=0)
    return y_true, q_true, cwv_true, profiles_q, profiles_t, cwv    

## Running the Retrievals

In [ ]:
import numpy as np
ids = np.arange(3500)
rs = lview.map_async(run_retrieval, ids)

In [ ]:
from atms import create_output_file

root_group, v_y_true, v_cwv_true, v_cwv ,v_h2o = create_output_file("data/mcmc_retrievals_5.nc", 5, 15)
for y_true, h2o_true, cwv_true, profiles_q, profiles_t, cwv in rs:
    if not y_true is None:
        t = v_cwv_true.shape[0]
        print("saving simulation: " + str(t))
        steps=cwv.size
        v_y_true[t,:]      = y_true
        ws.vmr_field.value[0,:,:,:] = h2o_true.reshape(-1,1,1)
        v_cwv_true[t]      = cwv_true
        v_cwv[t, :steps]   = cwv[:]
        v_h2o[t, :steps,:] = profiles_q.ravel().reshape(-1, 15)
    else:
        print("failure in simulation: " + str(t))
        print(h2o_true)
        print(cwv_true)
        print(profiles)

loading existing file: data/mcmc_retrievals_5.nc
saving simulation: 9046
saving simulation: 9047
saving simulation: 9048
saving simulation: 9049
saving simulation: 9050
saving simulation: 9051
saving simulation: 9052
saving simulation: 9053
saving simulation: 9054
saving simulation: 9055
saving simulation: 9056
saving simulation: 9057
saving simulation: 9058
saving simulation: 9059
saving simulation: 9060
saving simulation: 9061
saving simulation: 9062
saving simulation: 9063
saving simulation: 9064
saving simulation: 9065
saving simulation: 9066
saving simulation: 9067
saving simulation: 9068
saving simulation: 9069
saving simulation: 9070
saving simulation: 9071
saving simulation: 9072
saving simulation: 9073
saving simulation: 9074
saving simulation: 9075
saving simulation: 9076
saving simulation: 9077
saving simulation: 9078
saving simulation: 9079
saving simulation: 9080
saving simulation: 9081
saving simulation: 9082
saving simulation: 9083
saving simulation: 9084
saving simulati

saving simulation: 9388
saving simulation: 9389
saving simulation: 9390
saving simulation: 9391
saving simulation: 9392
saving simulation: 9393
saving simulation: 9394
saving simulation: 9395
saving simulation: 9396
saving simulation: 9397
saving simulation: 9398
saving simulation: 9399
saving simulation: 9400
saving simulation: 9401
saving simulation: 9402
saving simulation: 9403
saving simulation: 9404
saving simulation: 9405
saving simulation: 9406
saving simulation: 9407
saving simulation: 9408
saving simulation: 9409
saving simulation: 9410
saving simulation: 9411
saving simulation: 9412
saving simulation: 9413
saving simulation: 9414
saving simulation: 9415
saving simulation: 9416
saving simulation: 9417
saving simulation: 9418
saving simulation: 9419
saving simulation: 9420
saving simulation: 9421
saving simulation: 9422
saving simulation: 9423
saving simulation: 9424
saving simulation: 9425
saving simulation: 9426
saving simulation: 9427
saving simulation: 9428
saving simulatio

saving simulation: 9730
saving simulation: 9731
saving simulation: 9732
saving simulation: 9733
saving simulation: 9734
saving simulation: 9735
saving simulation: 9736
saving simulation: 9737
saving simulation: 9738
saving simulation: 9739
saving simulation: 9740
saving simulation: 9741
saving simulation: 9742
saving simulation: 9743
saving simulation: 9744
saving simulation: 9745
saving simulation: 9746
saving simulation: 9747
saving simulation: 9748
saving simulation: 9749
saving simulation: 9750
saving simulation: 9751
saving simulation: 9752
saving simulation: 9753
saving simulation: 9754
saving simulation: 9755
saving simulation: 9756
saving simulation: 9757
saving simulation: 9758
saving simulation: 9759
saving simulation: 9760
saving simulation: 9761
saving simulation: 9762
saving simulation: 9763
saving simulation: 9764
saving simulation: 9765
saving simulation: 9766
saving simulation: 9767
saving simulation: 9768
saving simulation: 9769
saving simulation: 9770
saving simulatio

saving simulation: 10071
saving simulation: 10072
saving simulation: 10073
saving simulation: 10074
saving simulation: 10075
saving simulation: 10076
saving simulation: 10077
saving simulation: 10078
saving simulation: 10079
saving simulation: 10080
saving simulation: 10081
saving simulation: 10082
saving simulation: 10083
saving simulation: 10084
saving simulation: 10085
saving simulation: 10086
saving simulation: 10087
saving simulation: 10088
saving simulation: 10089
saving simulation: 10090
saving simulation: 10091
saving simulation: 10092
saving simulation: 10093
saving simulation: 10094
saving simulation: 10095
saving simulation: 10096
saving simulation: 10097
saving simulation: 10098
saving simulation: 10099
saving simulation: 10100
saving simulation: 10101
saving simulation: 10102
saving simulation: 10103
saving simulation: 10104
saving simulation: 10105
saving simulation: 10106
saving simulation: 10107
saving simulation: 10108
saving simulation: 10109
saving simulation: 10110


saving simulation: 10403
saving simulation: 10404
saving simulation: 10405
saving simulation: 10406
saving simulation: 10407
saving simulation: 10408
saving simulation: 10409
saving simulation: 10410
saving simulation: 10411
saving simulation: 10412
saving simulation: 10413
saving simulation: 10414
saving simulation: 10415
saving simulation: 10416
saving simulation: 10417
saving simulation: 10418
saving simulation: 10419
saving simulation: 10420
saving simulation: 10421
saving simulation: 10422
saving simulation: 10423
saving simulation: 10424
saving simulation: 10425
saving simulation: 10426
saving simulation: 10427
saving simulation: 10428
saving simulation: 10429
saving simulation: 10430
saving simulation: 10431
saving simulation: 10432
saving simulation: 10433
saving simulation: 10434
saving simulation: 10435
saving simulation: 10436
saving simulation: 10437
saving simulation: 10438
saving simulation: 10439
saving simulation: 10440
saving simulation: 10441
saving simulation: 10442


saving simulation: 10731
saving simulation: 10732
saving simulation: 10733
saving simulation: 10734
saving simulation: 10735
saving simulation: 10736
saving simulation: 10737
saving simulation: 10738
saving simulation: 10739
saving simulation: 10740
saving simulation: 10741
saving simulation: 10742
saving simulation: 10743
saving simulation: 10744
saving simulation: 10745
saving simulation: 10746
saving simulation: 10747
saving simulation: 10748
saving simulation: 10749
saving simulation: 10750
saving simulation: 10751
saving simulation: 10752
saving simulation: 10753
saving simulation: 10754
saving simulation: 10755
saving simulation: 10756
saving simulation: 10757
saving simulation: 10758
saving simulation: 10759
saving simulation: 10760
saving simulation: 10761
saving simulation: 10762
saving simulation: 10763
saving simulation: 10764
saving simulation: 10765
saving simulation: 10766
saving simulation: 10767
saving simulation: 10768
saving simulation: 10769
saving simulation: 10770


saving simulation: 11060
saving simulation: 11061
saving simulation: 11062
saving simulation: 11063
saving simulation: 11064
saving simulation: 11065
saving simulation: 11066
saving simulation: 11067
saving simulation: 11068
saving simulation: 11069
saving simulation: 11070
saving simulation: 11071
saving simulation: 11072
saving simulation: 11073
saving simulation: 11074
saving simulation: 11075
saving simulation: 11076
saving simulation: 11077
saving simulation: 11078
saving simulation: 11079
saving simulation: 11080
saving simulation: 11081
saving simulation: 11082
saving simulation: 11083
saving simulation: 11084
saving simulation: 11085
saving simulation: 11086
saving simulation: 11087
saving simulation: 11088
saving simulation: 11089
saving simulation: 11090
saving simulation: 11091
saving simulation: 11092
saving simulation: 11093
saving simulation: 11094
saving simulation: 11095
saving simulation: 11096
saving simulation: 11097
saving simulation: 11098
saving simulation: 11099


saving simulation: 11389
saving simulation: 11390
saving simulation: 11391
saving simulation: 11392
saving simulation: 11393
saving simulation: 11394
saving simulation: 11395
saving simulation: 11396
saving simulation: 11397
saving simulation: 11398
saving simulation: 11399
saving simulation: 11400
saving simulation: 11401
saving simulation: 11402
saving simulation: 11403
saving simulation: 11404
saving simulation: 11405
saving simulation: 11406
saving simulation: 11407
saving simulation: 11408
saving simulation: 11409
saving simulation: 11410
saving simulation: 11411
saving simulation: 11412
saving simulation: 11413
saving simulation: 11414
saving simulation: 11415
saving simulation: 11416
saving simulation: 11417
saving simulation: 11418
saving simulation: 11419
saving simulation: 11420
saving simulation: 11421
saving simulation: 11422
saving simulation: 11423
saving simulation: 11424
saving simulation: 11425
saving simulation: 11426
saving simulation: 11427
saving simulation: 11428


saving simulation: 11721
saving simulation: 11722
saving simulation: 11723
saving simulation: 11724
saving simulation: 11725
saving simulation: 11726
saving simulation: 11727
saving simulation: 11728
saving simulation: 11729
saving simulation: 11730
saving simulation: 11731
saving simulation: 11732
saving simulation: 11733
saving simulation: 11734
saving simulation: 11735
saving simulation: 11736
saving simulation: 11737
saving simulation: 11738
saving simulation: 11739
saving simulation: 11740
saving simulation: 11741
saving simulation: 11742
saving simulation: 11743
saving simulation: 11744
saving simulation: 11745
saving simulation: 11746
saving simulation: 11747
saving simulation: 11748
saving simulation: 11749
saving simulation: 11750
saving simulation: 11751
saving simulation: 11752
saving simulation: 11753
saving simulation: 11754
saving simulation: 11755
saving simulation: 11756
saving simulation: 11757
saving simulation: 11758
saving simulation: 11759
saving simulation: 11760


In [ ]:
import matplotlib_settings
import matplotlib.pyplot as plt

In [16]:
root_group.close()

In [ ]:

root_group, v_y_true, v_cwv_true, v_cwv ,v_h2o = create_output_file("data/mcmc_retrievals_5.nc", 5, 27)

In [ ]:
for i in range(1000, 1100):
    plt.plot(v_cwv[i, :])
    plt.gca().axhline(v_cwv_true[i], c = 'k', ls = '--')

In [ ]:
v_h2o[118, 250:500, :].shape

In [ ]:
plt.plot(np.mean(profs_t[2, 0:200], axis = 0), p)
plt.plot(np.mean(profs_t[2, 200:400], axis = 0), p)
plt.title("Temperature Profiles")
plt.xlabel("T [K]")
plt.ylabel("P [hPa]")
plt.gca().invert_yaxis()

In [ ]:
p = np.load("data/p_grid.npy")
profiles_t[1, :, :].shape

In [ ]:
plt.plot(np.mean(np.exp(profs_q[1, 0:200]) * 18.0 / 28.9, axis = 0), p)
plt.plot(np.mean(np.exp(profs_q[1, 200:400]) * 18.0/ 28.9, axis = 0), p)
plt.gca().invert_yaxis()
plt.title("Water Vapor Profiles")
plt.xlabel("$H_2O$ [mol / mol]")
plt.ylabel("P [hPa]")